- Führt Support Vector Machine Klassifikatoin durch 
- Der Klassifikator wurde optimiert mit Grid search : kernel ='sigmoid', probability = True, C=2, degree = 1, gamma = 1
- Erstellt eine Pickle datei

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
import csv
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import pickle
import os

### Definiere hier deinen eigenen Datenpfad!!

In [5]:
#Input Datenpfad (Ordner mit train und test Daten)
Input_Datenpfad = 'C:/Users/pasca/OneDrive/Dokumente/MALE/KaggleParlamentStGallen/DonwloadDatenKaggle'
Input_Datenpfad = r'C:\Users\TOB\OneDrive\OneDrive - OST\Projekte_Onedrive\MaLe-Entwicklung\Überarbeitung FS22\Nicola_Wullschleger\opendatasg\input'
Testdatei = os.path.join(Input_Datenpfad,'test.csv')
Trainingsdatei = os.path.join(Input_Datenpfad,'train.csv')

#Output Datenpfad (Ordner wo die Pickle Datei und die Kaggle Submission gespeicher werden)
Output_Datenpfad = 'C:/Users/pasca/OneDrive/Dokumente/MALE/KaggleParlamentStGallen/ParlamentStGallenSVM_Classifier'
Output_Datenpfad = r'C:\Users\TOB\OneDrive\OneDrive - OST\Projekte_Onedrive\MaLe-Entwicklung\Überarbeitung FS22\Nicola_Wullschleger\opendatasg\output'

Ausgabe_Submission = os.path.join(Output_Datenpfad,'Sample_submission_pw.csv')
Ausgabe_Pickle_Datei = os.path.join(Output_Datenpfad,'SVMStGallen.pickle')

### Datenimport

In [6]:
dftest = pd.read_csv(Testdatei,index_col='Id')
dftrain = pd.read_csv(Trainingsdatei,index_col='Id')

### Vectorizer

In [7]:
sw = set(stopwords.words('german'))
vect = TfidfVectorizer(stop_words=sw)
Xtrain = vect.fit_transform(dftrain.X1)
Xtest = vect.transform(dftest.X1)

### Model Fit

In [8]:
clf = svm.SVC(kernel='sigmoid',probability=True,C=2,degree=1,gamma=1)
clf.fit(Xtrain, dftrain.y)
clf.score(Xtrain,dftrain.y)

0.9850498338870431

### Model Preditct und Transformation in Format für Kaggle Submission

In [9]:
phat_test = clf.predict_proba(vect.transform(dftest.X1))
indices = np.argsort(phat_test,axis=1)[:,-3:]
indices=indices[:,::-1]
rowlist = [str(ind)+','+' '.join(row)+'\n' for row,ind in zip(indices.astype('str').tolist(),dftest.index)]
rowlist[:5]

['0,16 2 4\n', '1,12 11 2\n', '2,0 16 6\n', '3,16 14 5\n', '4,10 8 16\n']

### Speichern CSV

In [10]:
with open(Ausgabe_Submission,'w') as fh:
    fh.write('Id,y\n')
    fh.writelines(rowlist)

### Erstellen und Export von Pickle Datei

In [11]:
SVMpipStGallen = Pipeline([('tfidf',TfidfVectorizer(stop_words=sw)),
                           ('classifier',svm.SVC(kernel='sigmoid',probability=True,C=2,degree=1,gamma=1))])

In [12]:
SVMpipStGallen.fit(dftrain.X1,dftrain.y)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(stop_words={'aber', 'alle', 'allem', 'allen',
                                             'aller', 'alles', 'als', 'also',
                                             'am', 'an', 'ander', 'andere',
                                             'anderem', 'anderen', 'anderer',
                                             'anderes', 'anderm', 'andern',
                                             'anderr', 'anders', 'auch', 'auf',
                                             'aus', 'bei', 'bin', 'bis', 'bist',
                                             'da', 'damit', 'dann', ...})),
                ('classifier',
                 SVC(C=2, degree=1, gamma=1, kernel='sigmoid',
                     probability=True))])

In [13]:
with open(Ausgabe_Pickle_Datei,'wb') as fh:
    pickle.dump(SVMpipStGallen,fh)

### Import und Testen von Pickle

In [53]:
with open(Ausgabe_Pickle_Datei, 'rb') as fh:
    SVMStGallen = pickle.load(fh)

In [54]:
phat_test = SVMStGallen.predict_proba(dftest.X1)
indices = np.argsort(phat_test,axis=1)[:,-3:]
indices=indices[:,::-1]
rowlist = [str(ind)+','+' '.join(row)+'\n' for row,ind in zip(indices.astype('str').tolist(),dftest.index)]
rowlist[:5]

['0,16 2 4\n', '1,12 11 13\n', '2,0 16 6\n', '3,16 14 5\n', '4,10 8 16\n']

Ausgabe soll eine Zele mit folgenden Argumenten enthalten: default das ein Titel ausgegeben wird, aber auch einstellung das alle 3 oder eine beliebige Anzahl an Titeln ausgegeben wird. Hinter dem Titel soll die Proba stehen. das heisst, ein panda df mit zwei spalten. wenn man drei ausgaben haben will dann gibt es hald 6 Spalten